In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = '../data/cifar-10-batches-py'
print("1. Structure of CIFAR_DIR {0}".format(os.listdir(CIFAR_DIR)))

1. Structure of CIFAR_DIR ['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def loadData(filename):
    '''
    Read data from filename
    '''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    '''
    CifarData is a wrapper that process CIFAR-10 Data
    '''
    def __init__(self, filenames, need_shuffle):
        '''
        The constructor takes in a list of files and read all data and labels into self._data member
        @param filenames - list of files that contains all data
        @param need_shuffle - a boolean flag indicating whether order of data needs to be shuffled. 
                              `True` of training set and `False` for testing set
        '''
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = loadData(filename)
            all_data.append(data)
            all_labels.append(labels)
        
        self._data = np.vstack(all_data) / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
            # When we process the data by Mini Batch, this is a cursor indicating where
            # we are in the dataset
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        '''
        Get a random permutation of all indices between 0 and _num_examples, and apply 
        this random permutation of indices to `self._data` and `self._labels`.
        '''
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        '''
        Generate a batch of data for processing. For training set, if end index is larger 
        than `self._num_examples`, reshuffle the data and start over from index 0. For 
        testing set, if end index is out of bound, raise an exception.
        @param batch_size - Number of samples for the batch of data and labels to return
        '''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("Do not have enough testing samples for next batch")
                
        if end_indicator > self._num_examples:
            raise Exception("Batch size is larger than _num_examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def reset(self):
        '''
        `_shuffle_data` and reset `_indicator` to 0
        '''
        self._shuffle_data()
        self._indicator = 0
    
    def getDataShape(self):
        return self._data.shape
    
    def getLabelShape(self):
        return self._labels.shape
    
print("2. Basic test for `loadData` and `CifarData` class.")
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
print("Train data shape = {0}, labels shape = {1}".
      format(train_data.getDataShape(), train_data.getLabelShape()))

test_data = CifarData(test_filenames, False)
print("Test data shape = {0}, labels shape = {1}".
      format(test_data.getDataShape(), test_data.getLabelShape()))

batch_data, batch_labels = train_data.next_batch(10)
print("Batch Data = {0}, Batch Labels = {1}".format(batch_data, batch_labels))

2. Basic test for `loadData` and `CifarData` class.
Train data shape = (50000, 3072), labels shape = (50000,)
Test data shape = (10000, 3072), labels shape = (10000,)
Batch Data = [[ 0.85882353  0.78039216  0.70980392 ...  0.03529412  0.02745098
   0.03529412]
 [-0.71764706 -0.73333333 -0.70196078 ...  0.79607843  0.78823529
   0.79607843]
 [-0.45098039 -0.45882353 -0.4745098  ...  0.69411765  0.71764706
   0.70980392]
 ...
 [ 0.15294118  0.16862745  0.23137255 ...  0.61568627  0.63137255
   0.75686275]
 [ 0.8745098   0.89803922  0.92941176 ... -0.57647059 -0.63137255
  -0.56078431]
 [ 1.          0.96078431  0.95294118 ...  0.89803922  0.9372549
   0.96862745]], Batch Labels = [1 4 7 9 5 4 2 0 9 3]


In [3]:
# Define x and y
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# Define weights w and intercept b
# w shape is (3072, 10) and it is initialized as a randome value distributed normally
# b shape is (10,) and all of them are initialized as 0.0
w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer = tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [10], initializer = tf.constant_initializer(0.0))

# y_pred, predict and y_actual are both [None, 10] shape
z = tf.matmul(x, w) + b
y_prob = tf.nn.softmax(z)
y_predict = tf.argmax(z, 1)
y_actual_oneHot = tf.one_hot(y, 10, dtype = tf.float32)

# Calculate loss and accuracy
loss = tf.reduce_mean(tf.square(y_actual_oneHot - y_prob))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_predict, y), tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
print("3. Apply TensorFlow model to CIFAR-10 Data.")
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

3. Apply TensorFlow model to CIFAR-10 Data.
[Train] step: 500, loss: 0.18627, accuracy: 0.00000
[Train] step: 1000, loss: 0.18862, accuracy: 0.05000
[Train] step: 1500, loss: 0.12322, accuracy: 0.35000
[Train] step: 2000, loss: 0.15539, accuracy: 0.20000
[Train] step: 2500, loss: 0.12072, accuracy: 0.35000
[Train] step: 3000, loss: 0.12562, accuracy: 0.35000
[Train] step: 3500, loss: 0.14823, accuracy: 0.25000
[Train] step: 4000, loss: 0.15569, accuracy: 0.20000
[Train] step: 4500, loss: 0.14602, accuracy: 0.25000
[Train] step: 5000, loss: 0.13721, accuracy: 0.30000
[Test ] step: 5000, accuracy: 0.30700
[Train] step: 5500, loss: 0.14269, accuracy: 0.25000
[Train] step: 6000, loss: 0.14821, accuracy: 0.25000
[Train] step: 6500, loss: 0.15958, accuracy: 0.20000
[Train] step: 7000, loss: 0.12183, accuracy: 0.35000
[Train] step: 7500, loss: 0.12398, accuracy: 0.35000
[Train] step: 8000, loss: 0.11998, accuracy: 0.40000
[Train] step: 8500, loss: 0.17885, accuracy: 0.10000
[Train] step: 9000

[Train] step: 71500, loss: 0.12897, accuracy: 0.35000
[Train] step: 72000, loss: 0.14048, accuracy: 0.30000
[Train] step: 72500, loss: 0.11573, accuracy: 0.40000
[Train] step: 73000, loss: 0.14267, accuracy: 0.25000
[Train] step: 73500, loss: 0.12742, accuracy: 0.35000
[Train] step: 74000, loss: 0.12664, accuracy: 0.30000
[Train] step: 74500, loss: 0.10686, accuracy: 0.45000
[Train] step: 75000, loss: 0.11898, accuracy: 0.40000
[Test ] step: 75000, accuracy: 0.38150
[Train] step: 75500, loss: 0.09607, accuracy: 0.45000
[Train] step: 76000, loss: 0.12572, accuracy: 0.35000
[Train] step: 76500, loss: 0.13785, accuracy: 0.30000
[Train] step: 77000, loss: 0.13802, accuracy: 0.30000
[Train] step: 77500, loss: 0.13679, accuracy: 0.25000
[Train] step: 78000, loss: 0.13349, accuracy: 0.30000
[Train] step: 78500, loss: 0.09891, accuracy: 0.50000
[Train] step: 79000, loss: 0.10870, accuracy: 0.45000
[Train] step: 79500, loss: 0.09952, accuracy: 0.50000
[Train] step: 80000, loss: 0.11135, accurac

In [5]:
print("4. After changing loss function to be cross entropy instead of mean square.")
loss_crossEntropy = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = z)

init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss_crossEntropy, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

4. After changing loss function to be cross entropy instead of mean square.
[Train] step: 500, loss: 37.12654, accuracy: 0.10000
[Train] step: 1000, loss: 36.20742, accuracy: 0.30000
[Train] step: 1500, loss: 36.90169, accuracy: 0.15000
[Train] step: 2000, loss: 24.93636, accuracy: 0.05000
[Train] step: 2500, loss: 30.49733, accuracy: 0.30000
[Train] step: 3000, loss: 29.01310, accuracy: 0.30000
[Train] step: 3500, loss: 33.96613, accuracy: 0.15000
[Train] step: 4000, loss: 27.40216, accuracy: 0.50000
[Train] step: 4500, loss: 40.58833, accuracy: 0.30000
[Train] step: 5000, loss: 29.47414, accuracy: 0.40000
[Test ] step: 5000, accuracy: 0.32700
[Train] step: 5500, loss: 31.14672, accuracy: 0.35000
[Train] step: 6000, loss: 38.12988, accuracy: 0.15000
[Train] step: 6500, loss: 36.90925, accuracy: 0.15000
[Train] step: 7000, loss: 40.09723, accuracy: 0.30000
[Train] step: 7500, loss: 42.78103, accuracy: 0.25000
[Train] step: 8000, loss: 41.93452, accuracy: 0.30000
[Train] step: 8500, los

[Train] step: 70000, loss: 35.52383, accuracy: 0.15000
[Test ] step: 70000, accuracy: 0.38050
[Train] step: 70500, loss: 34.78812, accuracy: 0.25000
[Train] step: 71000, loss: 27.74274, accuracy: 0.60000
[Train] step: 71500, loss: 30.75207, accuracy: 0.25000
[Train] step: 72000, loss: 19.19729, accuracy: 0.50000
[Train] step: 72500, loss: 29.33353, accuracy: 0.50000
[Train] step: 73000, loss: 45.59453, accuracy: 0.30000
[Train] step: 73500, loss: 20.71176, accuracy: 0.50000
[Train] step: 74000, loss: 49.94170, accuracy: 0.20000
[Train] step: 74500, loss: 28.93684, accuracy: 0.40000
[Train] step: 75000, loss: 34.97997, accuracy: 0.45000
[Test ] step: 75000, accuracy: 0.39550
[Train] step: 75500, loss: 33.45826, accuracy: 0.40000
[Train] step: 76000, loss: 25.10101, accuracy: 0.35000
[Train] step: 76500, loss: 12.71983, accuracy: 0.60000
[Train] step: 77000, loss: 35.62181, accuracy: 0.45000
[Train] step: 77500, loss: 41.20121, accuracy: 0.25000
[Train] step: 78000, loss: 33.98297, accur